# d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [ ]:
#filename의 경로의 파일을 읽어서 리턴
def read_data(filename):
    #filename에 해당하는 파일을 열어서 파일을 읽을수 있는 f를 생성해서 리턴
    with open(filename, 'r', encoding='utf8') as f:
        #f.read() : 파일 끝까지 읽음
        #splitlines() : 파일의 한줄씩 잘라서 리스트로 리턴
        #line 파일의 한줄이 저장되 있음
        #line.split("\t") : 파일의 한줄이 저장된 line에서 tab을 기준으로 잘라서 리스트로 리턴
        #탭을 기준으로 아이디,영화평,점수가 저장되 있음
        result = [line.split('\t') for line in f.read().splitlines() ]
        #result에 저장된 데이터에서 0번째 줄을 제외한 첫번째 줄부터 리턴
        #0번째 줄은 각 칸의 제목임
        result = result[1:] #header 제외
    return result

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_test.txt'

In [ ]:
#ratings_train.txt 파일을 읽어서 train_tmp 에 대입
train_tmp = read_data(train_path)

#ratings_test 파일을 읽어서 test_tmp 에 대입
test_tmp = read_data(test_path)

In [ ]:
train_tmp 

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1'],
 ['9008700', '걍인피니트가짱이다.진짜짱이다♥', '1'],
 ['10217543', '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~', '1'],
 ['5957425', '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해', '0'],
 ['8628627', '담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.', '1'],
 ['9864035', '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지', '0'],
 ['6852435', 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ', '1'],
 ['9143163',
  '참 사람들 웃긴게 

In [ ]:
test_tmp

[['6270596', '굳 ㅋ', '1'],
 ['9274899', 'GDNTOPCLASSINTHECLUB', '0'],
 ['8544678', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0'],
 ['6825595', '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0'],
 ['6723715', '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0'],
 ['7898805', '음악이 주가 된, 최고의 음악영화', '1'],
 ['6315043', '진정한 쓰레기', '0'],
 ['6097171', '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', '0'],
 ['8932678',
  '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
  '0'],
 ['6242223', '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..', '1'],
 ['7462111', '괜찮네요오랜만포켓몬스터잼밌어요', '1'],
 ['8425305', '한국독립영화의 한계 그렇게 아버지가 된다와 비교됨', '0'],
 ['6900881',
  '청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다.',
  '1'],
 ['9629375', '눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.', '1'],
 ['9037756',
  '"""스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해. 이런 영화 찍고도 김문옥 감독은 """"내가 영화 경력이 몇OO인데 조무래기들이 내 영화를 평론해?"""" 같은 마인드에 빠져있겠지?"""',
  '0'],
 ['10268521', '소위 ㅈ문가라는 평점은 뭐냐?', '1'],
 ['2

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 146 kB/s 
     |████████████████████████████████| 448 kB 47.1 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Twitter

In [ ]:
#한국어 문자열을 단어로 나누고 품사를 리턴하는 Twitter()객체 생성
twitter=Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
#안녕 하십니까 좋은 아침입니다. 반가워" 를 단어와 품사로 분리
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워")

[('안녕', 'Noun'),
 ('하십니까', 'Verb'),
 ('좋은', 'Adjective'),
 ('아침', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반가워', 'Adjective')]

In [ ]:
#안녕하세요 좋은 아침입니다. 반가워" 를 단어와 품사로 분리 
#stem=True : 동사인 하십니까 반가워는 기본 동사형 안녕하다와 반갑다 로 변환
twitter.pos("안녕하세요 좋은 아침입니다. 반가워",stem=True)

[('안녕하다', 'Adjective'),
 ('좋다', 'Adjective'),
 ('아침', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반갑다', 'Adjective')]

In [ ]:
#한글 품사 태깅을 할 객체
twitter = Twitter()
#영화 감상 평에서 명사와 형용사만 리턴하는 함수
#text: 영화 감상평 1줄이 대입되는 매개 변수
def getNounAdjective(text):
    #"" 가 40개인 리스트 생성
    stems = [""]*40
    #stems 의 마지막 인덱스를 stems_last_index에 저장
    stems_last_index = (len(stems)-1)
    #stems에 데이터를 추가할 인덱스
    stems_index=0

    #text(영화평이 대입될것임) 에서 단어와 품사를 리턴
    #예를 들어서 text="홍길동 간다"이면 
    #twitter.post(text,stem=True) 는 [["홍길동",Noun],["간다","Verb"]] 리턴해서 tagged에 저장
    tagged = twitter.pos(text, stem=True)
    #tagged의 줄수 만큼 반복
    for i in range (0, len(tagged)):
        #품사는 tagged의 1번째 칸에 저장
        #품사가 명사(Noun)  혹은 형용사 (Adjective) 일때
        if (tagged[i][1]=='Noun' or tagged[i][1]=='Adjective') :
            #stems에 단어 추가 단어는 tagged 각줄의 0번째 칸에 저장
            stems[stems_index]=tagged[i][0]
            #데이터를 추가 했으니 stems_index 1증가
            stems_index+=1
        
        #stems_index 가 stems_last_index 이면 반복 종료
        if stems_index == stems_last_index:
            break
    #stems_index는 리스트 stem에 저장된 단어의 개수임
    # stem에 저장된 단어가 2개 초과이면 stems 리턴
    if stems_index > 2 :    
        #리스트인 stems 리턴       
        return stems
    

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
#문장에서 명사와 형용사만 리턴됨
getNounAdjective("홍길동이 길을 걸어 갔다. 길동이 아버지는 방에 들어 가셨다. 그래서 길동은 슬퍼서 눈물을 흘렸다")

['홍길동',
 '길',
 '길동',
 '아버지',
 '방',
 '길동',
 '슬프다',
 '눈물',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
#문장에서 명사와 형용사만 리턴됨
#문장에 저장된 명사와 형용사 개수가 2이하이므로 리턴되는 데이터 None (없음)
getNounAdjective("홍길동 아버지")

In [ ]:
#반복문의 진행 상태를 나타내는 객체
from tqdm.notebook import tqdm

sum=0
#tqdm 을 사용하였기 때문에 반복문의 진행 상태를 
#진행 바로 나타냄
for i in tqdm(range(10000000)):
  sum=sum+i

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [ ]:
#학습데이터(train_tmp) 에서 ID를 삭제하고 리뷰데이터와 감성 라벨만 리턴
#테스트데이터(test_tmp) 에서 ID를 삭제하고 리뷰데이터와 감성 라벨만 리턴

def kor_movie():
    # 데이터 구조 : ID | 리뷰데이터 | 감성라벨
    # 필요없는 id를 제외하고 리뷰데이터과 감성라벨로만 이루어진 데이터를 만든다.
    # 학습 데이터의 리뷰 데이터를 저장할 리스트
    train_x = []
    #학습 데이터의 감성 라벨을 저장할 리스트
    train_y = []
    
    #len(train_tmp):학습데이터(train_tmp)의 줄의 수
    #학습 데이터의 줄의 수만큼 반복
    for i in tqdm(range(len(train_tmp))):
        #train_tmp[i][1]: 학습데이터의 i번째 줄의 1번째 칸 (영화 리뷰가 저장)을
        #getNounAdjective(): 함수를 실행-> 명사와 형용사만 리턴
        noun_adjective = getNounAdjective(train_tmp[i][1])
        
        #noun_adjective 가 None 이 아니면
        if noun_adjective is not None:
            #train_x에 명사와 형용사 추가
            train_x.append(noun_adjective)
            #train_tmp[i][1]: 학습데이터의 i번째 줄의 2번째 칸 (긍정 1 부정 0)을
            #int(): 정수로 변환 해서
            #train_y에 추가
            train_y.append(int(train_tmp[i][2]))

    # 테스트용
    test_x = []
    test_y = []
    #len(test_tmp):테스트 데이터(test_tmp)의 줄의 수
    #테스트 데이터의 줄의 수만큼 반복
    for i in tqdm(range(len(test_tmp))):
        #test_tmp[i][1]: 테스트 데이터의 i번째 줄의 1번째 칸 (영화 리뷰가 저장)을
        #getNounAdjective(): 함수를 실행-> 명사와 형용사만 리턴
        noun_adjective = getNounAdjective(test_tmp[i][1])
        #noun_adjective 가 None 이 아니면
        if noun_adjective is not None:
            #test_x에 명사와 형용사 추가
            test_x.append(noun_adjective)
            #test_tmp[i][1]: 테스트 데이터의 i번째 줄의 2번째 칸 (긍정 1 부정 0)을
            #int(): 정수로 변환 해서
            #test_y에 추가
            test_y.append(int(test_tmp[i][2]))
    #학습데이터(train_x:영화 리뷰, train_y: 긍정 (1) 부정 (0) 여부)
    #테스트 데이터(test_x:영화 리뷰, test_y: 긍정 (1) 부정 (0) 여부)
    return (train_x, train_y) , (test_x, test_y)

In [ ]:
print('...전처리하기...')
(X_train, y_train),(X_test,y_test) = kor_movie() 

...전처리하기...


  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
#메모리에서 train_tmp 삭제
del train_tmp

In [ ]:
#메모리에서 test_temp 삭제
del test_tmp

In [ ]:
#학습데이터를 조회
X_train

[['더빙',
  '진짜',
  '짜증나다',
  '목소리',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['흠',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '오버',
  '연기',
  '가볍다',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['무재',
  '밓었',
  '다그',
  '래서',
  '추천',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['교도소',
  '이야기',
  '구먼',
  '솔직하다',
  '재미',
  '없다',
  '평점',
  '조정',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [ ]:
#첫번재 리뷰부터 10개의 리뷰를 출력
X_train[0:10]

[['더빙',
  '진짜',
  '짜증나다',
  '목소리',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['흠',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '오버',
  '연기',
  '가볍다',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['무재',
  '밓었',
  '다그',
  '래서',
  '추천',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['교도소',
  '이야기',
  '구먼',
  '솔직하다',
  '재미',
  '없다',
  '평점',
  '조정',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [ ]:
from gensim.models import Word2Vec

In [ ]:
#X_train의 단어를 Word2Vector로 생성
# size=20 : 워드벡터로 변환한 데이터의 칸수
word2vector = Word2Vec(X_train, size=20)

In [ ]:
#word2vector.init_sims(replace=True) : Word2Vector를 계산할때 사용한 변수들을 메모리에서 삭제
word2vector.init_sims(replace=True)

In [ ]:
#영화와 가장 가까운 거리의 단어 조회
word2vector.wv.most_similar("영화")

[('영화로', 0.8424920439720154),
 ('무라카미', 0.7767491936683655),
 ('애니메이션', 0.7461503744125366),
 ('재난영화', 0.7455440759658813),
 ('공포물', 0.7322407364845276),
 ('가을동화', 0.7227115035057068),
 ('공포영화', 0.7208267450332642),
 ('만화영화', 0.7115954756736755),
 ('화란', 0.7113921046257019),
 ('애니', 0.7075731754302979)]

In [ ]:
#배우와 가장 가까운 거리의 단어 조회
word2vector.wv.most_similar("배우")

[('연기자', 0.9476362466812134),
 ('조연', 0.8877931833267212),
 ('배역', 0.8712071180343628),
 ('주연', 0.854213297367096),
 ('송강호', 0.8459872603416443),
 ('신인', 0.8366875052452087),
 ('차승원', 0.8277773857116699),
 ('안성기', 0.8253532648086548),
 ('외모', 0.8243128657341003),
 ('김창완', 0.8225927352905273)]

In [ ]:
#word2vector 가 벡터로 만든 단어 조회
word2vector.wv.vocab

{'더빙': <gensim.models.keyedvectors.Vocab at 0x7f5354c2d810>,
 '진짜': <gensim.models.keyedvectors.Vocab at 0x7f535a265b90>,
 '짜증나다': <gensim.models.keyedvectors.Vocab at 0x7f535a265750>,
 '목소리': <gensim.models.keyedvectors.Vocab at 0x7f5354ebab10>,
 '': <gensim.models.keyedvectors.Vocab at 0x7f5367c14ad0>,
 '흠': <gensim.models.keyedvectors.Vocab at 0x7f535a260410>,
 '포스터': <gensim.models.keyedvectors.Vocab at 0x7f5354bdd590>,
 '보고': <gensim.models.keyedvectors.Vocab at 0x7f5354bdd910>,
 '초딩': <gensim.models.keyedvectors.Vocab at 0x7f535c95b250>,
 '영화': <gensim.models.keyedvectors.Vocab at 0x7f5367c14dd0>,
 '줄': <gensim.models.keyedvectors.Vocab at 0x7f5354bdda90>,
 '오버': <gensim.models.keyedvectors.Vocab at 0x7f5354bdd990>,
 '연기': <gensim.models.keyedvectors.Vocab at 0x7f5354bdd950>,
 '가볍다': <gensim.models.keyedvectors.Vocab at 0x7f5354bddad0>,
 '무재': <gensim.models.keyedvectors.Vocab at 0x7f5354bdd890>,
 '다그': <gensim.models.keyedvectors.Vocab at 0x7f5354bddb50>,
 '래서': <gensim.models.k

In [ ]:
#배우의 벡터값 조회
word2vector["배우"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 0.01185523, -0.15960456,  0.19185   , -0.19377986, -0.06037734,
       -0.14665957, -0.08359569,  0.2341845 ,  0.02404744, -0.04222106,
        0.22095029, -0.39420843, -0.21075441,  0.50225645,  0.4097956 ,
        0.12697709,  0.21732622, -0.02985599,  0.25036064,  0.12217396],
      dtype=float32)

In [ ]:
#""의 벡터값 조회
word2vector[""]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-0.03500871,  0.36088094,  0.08279353, -0.12636705, -0.10819418,
       -0.00555942, -0.20104562,  0.08952411, -0.5526991 , -0.01348251,
        0.36969137, -0.35676765, -0.23197262,  0.09171937,  0.1320546 ,
        0.20936279,  0.13044913,  0.11336042,  0.08256648, -0.23611943],
      dtype=float32)

In [ ]:
#word2vector.wv.vocab.keys(): word2vector에 저장된 단어들을 Dictionary 객체로 리턴
word_vector_keys=word2vector.wv.vocab.keys()
word_vector_keys

dict_keys(['더빙', '진짜', '짜증나다', '목소리', '', '흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '무재', '다그', '래서', '추천', '교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정', '의', '익살스럽다', '스파이더맨', '이쁘다', '막', '걸음', '마', '세', '초등학교', '학년', '생인', '반개', '아깝다', '움', '원작', '긴장감', '제대로', '별', '욕', '이응경', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '드라마', '가족', '사람', '네', '액션', '있다', '몇', '안되다', '왜케', '낮다', '꽤', '볼', '데', '헐리우드', '화려하다', '인피니트', '짱', '볼때', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인', '울면', '손', '때', '뻔', '이범수', '드럽다', '담백하다', '깔끔하다', '좋다', '로만', '자꾸', '그', '것', '취향', '존중', '다지', '내생', '극장', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', '냥', '매번', '긴장', '재밋음', '바스코', '이기', '락스', '코', '바비', '아이돌', '그냥', '안달', '굿바이', '레닌', '표절', '이해', '왜', '뒤', '갈수록', '이건', '깨알', '캐스팅', '산뜻하다', '용구성', '일드', '위', '변명', '놈', '착하다', '절대', '아니다', '걸', '나름', '심오하다', '뜻', '듯', '학생', '선생', '재미없다', '지루하다', '같다', '음식', '도', '만찬', '별로', '핀란드', '풍경', '평범하다', '수작', '는걸', '주제', '중반', '납득', '수', '그렇다', '꼭', '

In [ ]:
#X_train의 문자열을 Word2Vector 로 변환해서 저장할 배열
X_train_vector=[]
#len(X_train) : X_train의 줄의 수만큼 반복
for index in tqdm(range(len(X_train))):
    input=[]
    #X_train[index]: X_train (네이버 영화 리뷰 단어가 저장된 리스트)
    #                     index번째줄 

    #단어를 word에 대입
    for word in X_train[index]:
        #단어 word가 word_vector_keys(Word2Vector에서 벡터로 변환한 단어 리스트) 에 포함되 있으면
        if word in word_vector_keys:
            #word2vector[word] : 단어의 word2vector 값을 input에 추가
            input.append(word2vector[word].tolist())
        else: 
            #word2vector[""] : ""의 word2vector 값을 input에 추가
            input.append(word2vector[""].tolist()) 

    X_train_vector.append(input)

  0%|          | 0/125008 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
X_train_vector[0]

[[-0.4010637104511261,
  -0.11653809249401093,
  0.12071069329977036,
  -0.3536963164806366,
  -0.06671569496393204,
  0.07308928668498993,
  -0.32086774706840515,
  -0.15966223180294037,
  0.13278552889823914,
  -0.30991294980049133,
  0.125550776720047,
  -0.06023705378174782,
  -0.20893947780132294,
  0.21698743104934692,
  0.17362794280052185,
  0.036315854638814926,
  0.37849223613739014,
  0.07434236258268356,
  0.20217764377593994,
  -0.32050007581710815],
 [0.06123073771595955,
  -0.001000958145596087,
  0.1720486581325531,
  -0.3627142012119293,
  -0.02312157303094864,
  0.0994146466255188,
  -0.2106204628944397,
  -0.07505635172128677,
  -0.3756900131702423,
  -0.17696194350719452,
  0.19501052796840668,
  -0.16788095235824585,
  -0.10669679939746857,
  0.09059587121009827,
  0.14599649608135223,
  0.009687421843409538,
  0.6577977538108826,
  -0.08478888869285583,
  -0.07709211111068726,
  -0.221748948097229],
 [-0.2550930976867676,
  0.0706978291273117,
  0.1073831915855407

In [ ]:
#X_train_vector 를 numpy 배열로 변환
X_train_vector=np.array(X_train_vector, dtype="float32")

In [ ]:
del X_train

In [ ]:
#X_test 의 문자열을 Word2Vector 로 변환해서 저장할 배열
X_test_vector=[]
#len(X_test) : X_test의 줄의 수만큼 반복
for index in tqdm(range(len(X_test))):
    input=[]
    #X_test[index]: X_test (네이버 영화 리뷰 단어가 저장된 리스트)
    #                     index번째줄 

    #단어를 word에 대입
    for word in X_test[index]:
        #단어 word가 word_vector_keys(Word2Vector에서 벡터로 변환한 단어 리스트) 에 포함되 있으면
        if word in word_vector_keys:
            #word2vector[word] : 단어의 word2vector 값을 input에 추가
            input.append(word2vector[word].tolist())
        else: 
            #word2vector[""] : ""의 word2vector 값을 input에 추가
            input.append(word2vector[""].tolist()) 

    X_test_vector.append(input)

  0%|          | 0/41727 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
#메모리에서 X_test 삭제
del X_test

In [ ]:
from tensorflow.keras import Sequential
#예측을 실행할 모델 객체 생성
model = Sequential()

In [ ]:
from tensorflow.keras.layers import LSTM

#LSTM 실행 객체
model.add(LSTM(units=128, input_shape=(40,20)))

In [ ]:
from tensorflow.keras.layers import Dense
#결과의 확률을 계산할 레이어 0(부정),1(긍정) 2개의 값중 하나가 리턴됨
model.add(Dense(1,activation="sigmoid"))

In [ ]:
#model의 정보를 조회
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               76288     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 76,417
Trainable params: 76,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import Adam
#모델을 실행할 준비를 함
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=1e-3),
            metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#y_train을 float32 타입으로 변환
y_train=np.array(y_train, dtype="float32")

In [ ]:
y_train

array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)

In [ ]:
#학습 시작
model.fit(X_train_vector, y_train,
          batch_size=1000,
          epochs=100)

Epoch 1/100
126/126 [==============================] - 15s 33ms/step - loss: 0.5216 - acc: 0.7194
Epoch 2/100
126/126 [==============================] - 4s 32ms/step - loss: 0.4517 - acc: 0.7921
Epoch 3/100
126/126 [==============================] - 5s 41ms/step - loss: 0.4427 - acc: 0.7948
Epoch 4/100
126/126 [==============================] - 4s 32ms/step - loss: 0.4375 - acc: 0.7961
Epoch 5/100
126/126 [==============================] - 4s 32ms/step - loss: 0.4440 - acc: 0.7927
Epoch 6/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4310 - acc: 0.7991
Epoch 7/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4460 - acc: 0.7923
Epoch 8/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4274 - acc: 0.8017
Epoch 9/100
126/126 [==============================] - 4s 32ms/step - loss: 0.4268 - acc: 0.8021
Epoch 10/100
126/126 [==============================] - 4s 32ms/step - loss: 0.4224 - acc: 0.8042
Epoch 11/100
126/126 [======

In [ ]:
X_test_vector[0]

[[-0.12288953363895416,
  0.10275613516569138,
  0.3767738938331604,
  -0.31405243277549744,
  0.11420563608407974,
  -0.2966897785663605,
  -0.15706229209899902,
  -0.22574017941951752,
  -0.06679506599903107,
  -0.15246543288230896,
  -0.21134433150291443,
  -0.06318651139736176,
  0.06491083651781082,
  0.044998910278081894,
  0.25640368461608887,
  -0.011653713881969452,
  0.47846415638923645,
  -0.13227222859859467,
  -0.12940074503421783,
  -0.3813205361366272],
 [-0.011665444821119308,
  0.1494111716747284,
  0.15048962831497192,
  -0.2688397765159607,
  -0.030726490542292595,
  0.21710515022277832,
  -0.0852024108171463,
  0.13495513796806335,
  -0.22526249289512634,
  -0.332856684923172,
  0.020632324740290642,
  -0.4416402578353882,
  0.054653797298669815,
  0.1557309329509735,
  -0.009905695915222168,
  0.6136280298233032,
  0.02017166092991829,
  -0.12616793811321259,
  0.033450622111558914,
  -0.1753215789794922],
 [-0.12190376222133636,
  0.015576156787574291,
  -0.312755

In [ ]:
y_test

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,


In [ ]:
#y_test를 float32타입으로 변환
y_test=np.array(y_test, dtype="float32")

In [ ]:
X_test_vector

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#X_test_vector를 float32타입으로 변환
X_test_vector=np.array(X_test_vector, dtype="float32")

In [ ]:
X_test_vector

array([[[-0.12288953,  0.10275614,  0.3767739 , ..., -0.13227223,
         -0.12940075, -0.38132054],
        [-0.01166544,  0.14941117,  0.15048963, ..., -0.12616794,
          0.03345062, -0.17532158],
        [-0.12190376,  0.01557616, -0.31275576, ..., -0.0972045 ,
          0.09026631, -0.32357538],
        ...,
        [-0.03500871,  0.36088094,  0.08279353, ...,  0.11336042,
          0.08256648, -0.23611943],
        [-0.03500871,  0.36088094,  0.08279353, ...,  0.11336042,
          0.08256648, -0.23611943],
        [-0.03500871,  0.36088094,  0.08279353, ...,  0.11336042,
          0.08256648, -0.23611943]],

       [[-0.1294102 ,  0.23440921,  0.33660397, ..., -0.00479057,
         -0.25034222, -0.3470017 ],
        [-0.08075727,  0.05239418,  0.20988086, ..., -0.06312324,
          0.03398941, -0.13937093],
        [-0.12074304,  0.23797783,  0.15654266, ...,  0.01097202,
         -0.41895375, -0.0325513 ],
        ...,
        [-0.03500871,  0.36088094,  0.08279353, ...,  

In [ ]:
#X_test_vector(테스트 데이터) 의 cost와 정확도 조회
model.evaluate(X_test_vector, y_test)

1304/1304 [==============================] - 7s 5ms/step - loss: 0.4012 - acc: 0.8196


[0.4011814296245575, 0.8195652961730957]